<a href="https://colab.research.google.com/github/vanhuy1619/bigdata-frequence/blob/main/MMDS_Midterm_THENEWMENTORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
# !cp drive/MyDrive/MMDS-data/*.tgz .
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark as spark

print(spark.__version__)

3.1.1


# Prepare Data

In [ ]:
# ! cp /content/drive/MyDrive/MMDS-data/Midterm/*.csv .

In [ ]:
# ! ls *.csv

In [ ]:
# declare SparkContext
from pyspark import SparkContext

sc = SparkContext("local", "Midterm")
# sc.stop()

In [ ]:
# declare SQLContext
from pyspark.sql import SQLContext

sqlc = SQLContext(sc)

In [ ]:
df = sqlc.read.csv('/content/data.csv',
                   header=True, inferSchema=True)
df.show()


+-------------+----------+--------------------+----+-----+---+-----------+
|Member_number|      Date|     itemDescription|year|month|day|day_of_week|
+-------------+----------+--------------------+----+-----+---+-----------+
|         1249|01/01/2014|        citrus fruit|2014|    1|  1|          2|
|         1249|01/01/2014|              coffee|2014|    1|  1|          2|
|         1381|01/01/2014|                curd|2014|    1|  1|          2|
|         1381|01/01/2014|                soda|2014|    1|  1|          2|
|         1440|01/01/2014|    other vegetables|2014|    1|  1|          2|
|         1440|01/01/2014|              yogurt|2014|    1|  1|          2|
|         1659|01/01/2014| specialty chocolate|2014|    1|  1|          2|
|         1659|01/01/2014|   frozen vegetables|2014|    1|  1|          2|
|         1789|01/01/2014|      hamburger meat|2014|    1|  1|          2|
|         1789|01/01/2014|             candles|2014|    1|  1|          2|
|         1922|01/01/2014

In [ ]:
df.printSchema()

root
 |-- Member_number: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- itemDescription: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)



# Import Library

In [ ]:
from pyspark.sql.functions import when, array, array_contains, flatten, count, concat, col, lit, collect_list, regexp_replace, expr, coalesce, split, size
from pyspark.sql.types import StringType
from itertools import combinations
import math
import subprocess

# Add Function For Exercise 2 & 3

In [ ]:
# Hàm tổng cho reduceByKey
def add( x, y):
    return x + y

# Hàm loại bỏ các bản sao sau tích chéo
def remove_dup(record):
    x1, x2 = (record[0], record[1]) if isinstance(record[0], tuple) else ([record[0]], record[1])
    if not any(x == x2 for x in x1):
        a = sorted(list(x1) + [x2])
        return tuple(a)
    return x1

# Hàm kiểm tra xem hai tập hợp có chứa ít nhất một phần tử chung
def has_common_element(item_1, item_2):
    if len(item_1) > len(item_2):
        return all(any(k == l for k in item_1) for l in item_2)
    else:
        return all(any(k == l for k in item_2) for l in item_1)

# Hàm lọc cặp tập hợp
def filter_for_conf(item):
    return item if len(item[0][0]) > len(item[1][0]) and has_common_element(item[0][0], item[1][0]) else None

# Hàm tính Confidence
def calculate_confidence(item):
    parent = set(item[0][0])
    child = set([item[1][0]]) if isinstance(item[1][0], str) else set(item[1][0])
    parent_support = item[0][1]
    child_support = item[1][1]
    support = (parent_support / child_support) * 100
    return [list(child), list(parent.difference(child)), support]


def to_csv_frequent_pairs( frequent_pairs, file_path):
    if not frequent_pairs.isEmpty():
      data_DF  = frequent_pairs.toDF().withColumnRenamed("_1", "frequent_pairs").withColumnRenamed("_2", "count")
      out = data_DF.filter(size(data_DF["frequent_pairs"]) > 1)
      out = out.withColumn("frequent_pairs", col("frequent_pairs").cast(StringType()))
      out = out.withColumn("frequent_pairs", regexp_replace("frequent_pairs", "\[|\]", ""))

      data_DF = data_DF.select("frequent_pairs").filter(size(data_DF["frequent_pairs"]) > 1).withColumnRenamed("frequent_pairs", "items")
      # print(data_DF.rdd.collect())
      frequent_pair = data_DF.withColumn("items", col("items").cast(StringType()))
      frequent_pair = data_DF.withColumn("items", regexp_replace("items", "\[|\]", ""))
      print("Số lượng các cặp phổ biến: ", frequent_pair.count())
      def find_max_length_list(row1, row2):
        list1 = row1.items
        list2 = row2.items
        return row1 if len(list1) >= len(list2) else row2
      if not frequent_pair.rdd.isEmpty():
        print("Trong đó: ")
        max_length_row = data_DF.rdd.reduce(lambda x,y: x if len(x)>len(y) else y)
        for i in range(1,len(max_length_row.items)):
          frequent_pair_temp = data_DF.filter(size(data_DF.items) == i+1)
          print("\tSố lượng cặp phổ biến size ",i+1," là: ",frequent_pair_temp.count())

      print("\nBảng các cặp phổ biến")
      out.show(truncate=False)

      if os.path.exists(file_path):
          subprocess.call("rm -r {}".format(file_path), shell=True)
      # out.write.csv('pcy_frequent_pairs.csv', header=True, sep=";")
      out.select("frequent_pairs").coalesce(1).write.csv(file_path, header=True)
      print("Đã Lưu vào file {}\n\n".format(file_path))
    else:
      print("\nBảng các cặp phổ biến")
      print(frequent_pairs.take(10))
      print("Không thể lưu vào file {} vì không có frequent_pairs nào\n\n".format(file_path))

def to_csv_association_rules(associate_rules,file_path):

    if not associate_rules.isEmpty():
      data_DF  = associate_rules.toDF()\
                                    .withColumnRenamed("_1", "antecedent")\
                                    .withColumnRenamed("_2", "consequent")\
                                    .withColumnRenamed("_3", "confidence")
      out = data_DF.withColumn("antecedent", col("antecedent").cast(StringType()))
      out = out.withColumn("antecedent", regexp_replace("antecedent", "\[|\]", ""))
      out = out.withColumn("consequent", col("consequent").cast(StringType()))
      out = out.withColumn("consequent", regexp_replace("consequent", "\[|\]", ""))

      print("Bảng association rules: ")
      out.show(truncate=False)

      if os.path.exists(file_path):
          subprocess.call("rm -r {}".format(file_path), shell=True)
      # out.write.csv('{}', header=True, sep=";")
      out.coalesce(1).write.csv(file_path, header=True)
      print("Đã Lưu vào file {}\n\n".format(file_path))
    else:
      print("Bảng association rules: ")
      print(associate_rules.take(10))
      print("Không thể lưu vào file {} vì không có association nào\n\n".format(file_path))

# EXERCISE A) Câu 1

In [ ]:
# @title Câu 1: Giỏ hàng { run: "auto", vertical-output: true, form-width: "42%", display-mode: "form" }
grouped_df = df.groupBy("Member_number", "Date").agg(
                                                collect_list("itemDescription")
                                                .alias('itemDescription')
                                                )
# grouped_df.printSchema()
#loại bỏ giá trị trùng lặp
grouped_df = grouped_df.withColumn("itemDescription",
                                   expr("array_distinct(itemDescription)"))
grouped_df = grouped_df.withColumn("itemDescription",
                                   regexp_replace("itemDescription", "\[|\]", ""))
#order by member_number and date
grouped_df = grouped_df.orderBy("Member_number", "Date")
#show
grouped_df.printSchema()
grouped_df.show(truncate=False)

file_path = "baskets.csv"
if os.path.exists(file_path):
    subprocess.call("rm -r baskets.csv", shell=True)
grouped_df.coalesce(1).write.csv('baskets.csv', header=True, sep=";")
print("Đã Lưu vào file baskets.csv")


root
 |-- Member_number: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- itemDescription: string (nullable = false)

+-------------+----------+-------------------------------------------+
|Member_number|Date      |itemDescription                            |
+-------------+----------+-------------------------------------------+
|1000         |24/06/2014|whole milk,pastry,salty snack              |
|1001         |02/07/2014|sausage,whole milk,rolls/buns              |
|1002         |26/04/2014|butter,whole milk                          |
|1003         |19/01/2014|frozen meals,dental care                   |
|1003         |27/02/2014|rolls/buns                                 |
|1004         |02/04/2014|pip fruit,whole milk,tropical fruit        |
|1004         |05/01/2014|dish cleaner,cling film/bags               |
|1004         |07/01/2014|canned beer,frozen fish                    |
|1005         |23/01/2014|whipped/sour cream,margarine               |
|1006       

#EXERCISE B) Câu 2


In [ ]:
# @title Class APriori { display-mode: "code" }

class APriori:

    def __init__(self, path,s=0.005,c=0.3):

        self.confidences = c
        self.path = path
        self.minSupport = s

    def run(self):
      # sqlc = SQLContext(sc)
      data = sqlc.read.csv(self.path, header=True,sep=";")
      data = data.withColumnRenamed("itemDescription","items").select("items")
      dataRdd = data.rdd
      basketslen = dataRdd.count()
      # print(dataRdd.take(4))

      frequent_pairs = self.__find_frequent_pair(dataRdd, self.minSupport)
      to_csv_frequent_pairs(frequent_pairs, "apriori_frequent_pairs.csv")
      # print(frequent_pairs.collect())
      associate_rules = self.__find_association_rules(frequent_pairs, self.confidences, basketslen)
      to_csv_association_rules(associate_rules, "apriori_association_rules.csv")

    def __find_frequent_pair(self, dataRdd, minSupport):
        """  chuyển từ chuỗi qua mảng  """
        arrData = dataRdd.map(lambda line: line[0].split(','))
        # print(arrData.collect())

        """  flat về thành mảng đơn hết  """
        singleData = dataRdd.flatMap(lambda line: line[0].split(','))
        # print(singleData.collect())

        """  Unique frequent items trong datasets  """
        uniqueData = singleData.distinct()
        # print(uniqueData.collect())

        supportRdd = singleData.map(lambda item: (item, 1))\
                              .reduceByKey(add)
        # print(supportRdd.collect())
        supports = supportRdd.map(lambda item: item[1])
        # print(supports.collect())

        basketslen = arrData.count()

        """  Chọn ra các phần tử với minSupport  """
        supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport*basketslen)
        # print(supportRdd.collect())

        # Tạo ra các cặp phổ biến sẽ được cập nhật
        frequent_pairs = supportRdd.map(lambda item: ([item[0]], item[1]))
        # print(frequent_pairs.collect())
        supportRdd = supportRdd.map(lambda item: item[0])
        # print(supportRdd.collect())

        k_size = 2

        while not supportRdd.isEmpty():
          com1 = supportRdd.cartesian(uniqueData) # product 2 rdd lại
          # Loại bỏ các bản trùng lặp
          com1 = com1.map(lambda item: remove_dup(item)) \
                      .filter(lambda item: len(item) == k_size).distinct()

          com2 = com1.cartesian(arrData)
          com2 = com2.filter(lambda item: all(x in item[1] for x in item[0]))

          com2 = com2.map(lambda item: item[0]).map(lambda item: (item, 1))
          com2 = com2.reduceByKey(add)\
                     .filter(lambda item: item[1] >= minSupport*basketslen)

          frequent_pairs = frequent_pairs.union(com2)

          com2 = com2.map(lambda item: item[0])
          supportRdd = com2
          k_size = k_size + 1

        return frequent_pairs

    def __find_association_rules(self, frequent_pairs, confidence, basketslen):
      combined = frequent_pairs.cartesian(frequent_pairs)
      # combined.toDF().show(truncate=False)
      filter_combined = combined.filter(lambda item: filter_for_conf(item))
      # filter_combined.toDF().show(truncate=False)
      confidences = filter_combined.map(lambda item: calculate_confidence(item))
      confidences = confidences.filter(lambda x: x[2] > confidence*100)
      return confidences

In [ ]:
# @title Run APriori { vertical-output: true, form-width: "40%" }
min_support = 0.001 # @param {type:"number"}
min_confidence = .2 # @param {type:"number"}
# Tạo đối tượng Apriori
apriori = APriori("baskets.csv",min_support,min_confidence)
# Chạy thuật toán
apriori.run()

Số lượng các cặp phổ biến:  302
Trong đó: 
	Số lượng cặp phổ biến size  2  là:  302

Bảng các cặp phổ biến
+------------------------------+-----+
|frequent_pairs                |count|
+------------------------------+-----+
|pastry, whole milk            |13   |
|salty snack, whole milk       |5    |
|sausage, whole milk           |8    |
|rolls/buns, whole milk        |33   |
|butter, whole milk            |16   |
|pip fruit, whole milk         |11   |
|tropical fruit, whole milk    |10   |
|whipped/sour cream, whole milk|16   |
|canned beer, whole milk       |15   |
|margarine, whole milk         |12   |
|frozen fish, whole milk       |7    |
|candy, whole milk             |8    |
|frozen vegetables, whole milk |10   |
|brown bread, whole milk       |11   |
|napkins, whole milk           |10   |
|citrus fruit, whole milk      |8    |
|other vegetables, whole milk  |37   |
|bottled water, whole milk     |20   |
|UHT-milk, whole milk          |9    |
|coffee, whole milk            |5  

#EXERCISE C) Câu 3


In [ ]:
# @title Class PCY
from itertools import combinations
class PCY:
  def __init__(self, baskets_path, support=0.3, hash_table_size=10, confidence=0.5):
    self.baskets_path = baskets_path
    self.support = support
    self.hash_table_size = hash_table_size
    self.confidence = confidence
  def run(self):
    # sqlc = SQLContext(sc)
    df = sqlc.read.csv(self.baskets_path, header = True, sep=";")
    # a = self._get_frequent_pairs(df, self.support, self.hash_table_size)
    # print(a.collect())
    item_frequent, buckets_without_pairs, buckets = \
                self._get_candidate_pairs(df, self.support, self.hash_table_size)
    frequent_pairs = self._get_frequent_pairs(df, buckets_without_pairs, buckets, self.support)

    frequent_pairs = frequent_pairs.map(lambda x: (x[1],x[2])) \
                              .map(lambda line: ([x for x in line[0]],line[1]))
    item_frequent = item_frequent.map(lambda line: ([line[0]],line[1]))
    # print(len(frequent_pairs.collect()))
    # print(len(item_frequent.collect()))
    frequent_pairs_all_size = frequent_pairs.union(item_frequent)
    association_rules = \
                    self.__find_association_rules(frequent_pairs_all_size,
                                                  self.confidence, df.count())

    to_csv_frequent_pairs(frequent_pairs, "pcy_frequent_pairs.csv")
    to_csv_association_rules(association_rules, "pcy_association_rules.csv")

  def _get_candidate_pairs(self, df, support, hash_table_size):
    def combine_hashes(items, hash_table_size):
      combined_hash = 0
      for item in items:
          item_hash = hash(item)
          combined_hash += item_hash
      return combined_hash % hash_table_size
    rdd = df.select("itemDescription").rdd
    baskets = rdd.count()
    transactions = rdd.map(lambda row: set(row[0].split(",")))

    item_counts = transactions \
        .flatMap(lambda transaction: transaction) \
        .map(lambda item: (item, 1)) \
        .reduceByKey(lambda a, b: a + b)
    item_frequent = item_counts \
                    .filter(lambda item : item[1] / baskets >= support)
    # return item_frequent
    pairs = transactions \
        .flatMap(lambda transaction: [pair for r in range(2, len(transaction) + 1) for pair in combinations(transaction, r)]) \
        .map(lambda pair: (tuple(sorted(pair)), 1)) \
        .reduceByKey(lambda a, b: a + b)

    buckets = pairs \
        .map(lambda pair: (combine_hashes(pair[0], hash_table_size), pair[0], pair[1]))

    buckets_without_pairs = buckets.map(lambda pair: (pair[0], pair[2])) \
        .reduceByKey(lambda a, b: a + b)

    return item_frequent, buckets_without_pairs, buckets
  def _get_frequent_pairs(self, df, buckets_without_pairs, buckets, support):
    baskets = df.count()
    first_elements = buckets_without_pairs.filter(lambda bucket: bucket[1] / baskets >= support).map(lambda bucket: bucket[0])

    first_elements_set = set(first_elements.toLocalIterator())
    first_elements_broadcast = sc.broadcast(first_elements_set)

    candidate_pairs = buckets.filter(lambda bucket: bucket[0] in first_elements_broadcast.value)
    frequent_pairs = candidate_pairs.filter(lambda pair: pair[2] / baskets >= support)
    return frequent_pairs

  def __find_association_rules(self, frequent_pairs, confidence, basketslen):
    combined = frequent_pairs.cartesian(frequent_pairs)
    # combined.toDF().show(truncate=False)
    filter_combined = combined.filter(lambda item: filter_for_conf(item))
    # print(filter_combined.count())
    # filter_combined.toDF().show(truncate=False)
    confidences = filter_combined.map(lambda item: calculate_confidence(item))
    confidences = confidences.filter(lambda x: x[2] > confidence*100)
    return confidences

In [ ]:
# @title Run PCY { vertical-output: true, form-width: "40%" }
min_support = 0.001 # @param {type:"number"}
min_confidence = .2 # @param {type:"number"}
hash_size = 200 # @param {type:"number"}

pcy = PCY('./baskets.csv', min_support, hash_size, min_confidence)

pcy.run()

Số lượng các cặp phổ biến:  302
Trong đó: 
	Số lượng cặp phổ biến size  2  là:  302

Bảng các cặp phổ biến
+---------------------------------+-----+
|frequent_pairs                   |count|
+---------------------------------+-----+
|pastry, whole milk               |13   |
|salty snack, whole milk          |5    |
|rolls/buns, sausage              |5    |
|sausage, whole milk              |8    |
|rolls/buns, whole milk           |33   |
|butter, whole milk               |16   |
|pip fruit, whole milk            |11   |
|tropical fruit, whole milk       |10   |
|margarine, whipped/sour cream    |5    |
|bottled water, rolls/buns        |20   |
|bottled water, candy             |5    |
|candy, rolls/buns                |8    |
|bottled water, citrus fruit      |11   |
|citrus fruit, rolls/buns         |10   |
|bottled water, hard cheese       |6    |
|candy, whole milk                |8    |
|frozen vegetables, tropical fruit|6    |
|fruit/vegetable juice, rolls/buns|14   |
|beef, yogu

# EXERCISE D) Câu 4

In [ ]:
# @title Câu 4: FPGrowth with support="[0.0001, 0.001, 0.004, 0.005, 0.01]" { run: "auto", vertical-output: true, form-width: "40%" }
# @markdown With Confidence: 20%
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import size

grouped_df = sqlc.read.csv('/content/baskets.csv',
                   header=True, inferSchema=True, sep=";")

supportArr = [0.0001, 0.001, 0.004, 0.005, 0.01]
minConfidence = .2

for i in supportArr:
  print("\n\t\tTEST VỚI SUPPORT LÀ ",i,"\n")
  fp_growth = FPGrowth(minSupport=i, minConfidence=minConfidence)
  # grouped_df.show()
  grouped_df = grouped_df.withColumn("items", split(grouped_df.itemDescription, ","))
  model = fp_growth.fit(grouped_df)
  frequent_items = model.freqItemsets
  association_rules = model.associationRules

  frequent_pair = frequent_items.filter(size(frequent_items.items) > 1)
  frequent_pair = frequent_pair.withColumn("items", col("items").cast(StringType()))
  frequent_pair = frequent_pair.withColumn("items", regexp_replace("items", "\[|\]", ""))
  association_rules = association_rules \
                            .withColumn("antecedent",
                                      col("antecedent").cast(StringType()))
  association_rules = association_rules \
                            .withColumn("antecedent",
                                      regexp_replace("antecedent", "\[|\]", ""))
  association_rules = association_rules \
                            .withColumn("consequent",
                                      col("consequent").cast(StringType()))
  association_rules = association_rules \
                            .withColumn("consequent",
                                      regexp_replace("consequent", "\[|\]", ""))

  print("Số lượng các cặp phổ biến: ", frequent_pair.count())
  def find_max_length_list(row1, row2):
    list1 = row1.items
    list2 = row2.items
    return row1 if len(list1) >= len(list2) else row2
  if not frequent_pair.rdd.isEmpty():
    print("Trong đó: ")
    max_length_row = frequent_items.rdd.reduce(find_max_length_list)
    for i in range(1,len(max_length_row.items)):
      frequent_pair_temp = frequent_items.filter(size(frequent_items.items) == i+1)
      print("\tSố lượng cặp phổ biến size ",i+1," là: ",frequent_pair_temp.count())

  print("\nBảng các cặp phổ biến")
  frequent_pair.show(truncate = False)
  print("Bảng association rules")
  association_rules.show()
# frequent_pair.coalesce(1).write.csv('frequent_items_FP.csv', header=True)
# association_rules.coalesce(1).write.csv('association_rules_FP.csv', header=True)


		TEST VỚI SUPPORT LÀ  0.0001 

Số lượng các cặp phổ biến:  5105
Trong đó: 
	Số lượng cặp phổ biến size  2  là:  3054
	Số lượng cặp phổ biến size  3  là:  1726
	Số lượng cặp phổ biến size  4  là:  297
	Số lượng cặp phổ biến size  5  là:  24
	Số lượng cặp phổ biến size  6  là:  4

Bảng các cặp phổ biến
+------------------------------------------------------+----+
|items                                                 |freq|
+------------------------------------------------------+----+
|prosecco, specialty chocolate                         |1   |
|prosecco, specialty chocolate, rolls/buns             |1   |
|prosecco, specialty chocolate, white bread            |1   |
|prosecco, specialty chocolate, white bread, rolls/buns|1   |
|prosecco, rolls/buns                                  |1   |
|prosecco, pot plants                                  |1   |
|prosecco, pot plants, sausage                         |1   |
|prosecco, frankfurter                                 |1   |
|prosecco, sau

In [ ]:
# @title Câu 4: "Test With Parameter --- Compare with APriori and PCY" { vertical-output: true, form-width: "40%" }
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import size

grouped_df = sqlc.read.csv('/content/baskets.csv',
                   header=True, inferSchema=True, sep=";")

Enter_Min_Support = 0.001 # @param {type:"number"}
minConfidence = .2 # @param {type:"number"}

fp_growth = FPGrowth(minSupport=Enter_Min_Support, minConfidence=minConfidence)
# grouped_df.show()
grouped_df = grouped_df.withColumn("items", split(grouped_df.itemDescription, ","))
model = fp_growth.fit(grouped_df)
frequent_items = model.freqItemsets
association_rules = model.associationRules

frequent_pair_FP = frequent_items.filter(size(frequent_items.items) > 1)
frequent_pair = frequent_pair_FP.withColumn("items", col("items").cast(StringType()))
frequent_pair = frequent_pair.withColumn("items", regexp_replace("items", "\[|\]", ""))
association_rules = association_rules \
                          .withColumn("antecedent",
                                    col("antecedent").cast(StringType()))
association_rules = association_rules \
                          .withColumn("antecedent",
                                    regexp_replace("antecedent", "\[|\]", ""))
association_rules = association_rules \
                          .withColumn("consequent",
                                    col("consequent").cast (StringType()))
association_rules = association_rules \
                          .withColumn("consequent",
                                    regexp_replace("consequent", "\[|\]", ""))

print("Số lượng các cặp phổ biến: ", frequent_pair.count())
def find_max_length_list(row1, row2):
  list1 = row1.items
  list2 = row2.items
  return row1 if len(list1) >= len(list2) else row2
if not frequent_pair.rdd.isEmpty():
  print("Trong đó: ")
  max_length_row = frequent_items.rdd.reduce(find_max_length_list)
  for i in range(1,len(max_length_row.items)):
    frequent_pair_temp = frequent_items.filter(size(frequent_items.items) == i+1)
    print("\tSố lượng cặp phổ biến size ",i+1," là: ",frequent_pair_temp.count())

print("\nBảng các cặp phổ biến")
frequent_pair.show(truncate = False)
print("Bảng association rules")
association_rules.show()

frequent_pair_apriori = sqlc.read.csv('/content/apriori_frequent_pairs.csv',
                   header=True, inferSchema= True)
association_rules_apriori = sqlc.read.csv('/content/apriori_association_rules.csv',
                   header=True, inferSchema= True)\
                   .withColumnRenamed("antecedent", "antecedent1")\
                   .withColumnRenamed("consequent", "consequent1")\
                   .withColumnRenamed("confidence", "confidence1 (%)")
frequent_pair_pcy = sqlc.read.csv('/content/pcy_frequent_pairs.csv',
                   header=True, inferSchema= True)
association_rules_pcy = sqlc.read.csv('/content/pcy_association_rules.csv',
                   header=True, inferSchema= True)\
                   .withColumnRenamed("antecedent", "antecedent1")\
                   .withColumnRenamed("consequent", "consequent1")\
                   .withColumnRenamed("confidence", "confidence1 (%)")

frequent_pair_FP = frequent_pair_FP.select("items")
print("\t\t\t SO SÁNH FREQUENT ITEMSETS")

print("\n\t FPGrowth & APriori")
print("Bảng frequent_pair_apriori: ",frequent_pair_apriori.count())
frequent_pair_apriori.show(truncate=False)
print("Bảng frequent_pair_FPGrowth: ",frequent_pair_FP.count())
frequent_pair.show(truncate=False)
print("Subtrate của 2 bảng trên: ")

frequent_pair_FP_rdd = frequent_pair_FP.rdd.map(lambda row: sorted(row[0])).sortBy(lambda x: x)
frequent_pair_apriori = frequent_pair_apriori.withColumn("frequent_pairs", split(frequent_pair_apriori.frequent_pairs, ", "))
frequent_pair_apriori = frequent_pair_apriori.rdd.map(lambda row: sorted(row[0])).sortBy(lambda x: x)

substrate_rdd = frequent_pair_FP_rdd.map(tuple).subtract(frequent_pair_apriori.map(tuple))
if not substrate_rdd.isEmpty():
  frequent_pair_FP_rdd.map(tuple).subtract(frequent_pair_apriori.map(tuple)).toDF().show()
print(frequent_pair_FP_rdd.map(tuple).subtract(frequent_pair_apriori.map(tuple)).take(20))

print("\n\t FPGrowth & PCY")
print("Bảng frequent_pair_pcy: ")
frequent_pair_pcy.show(truncate=False)
print("Bảng frequent_pair_FPGrowth: ",frequent_pair_FP.count())
frequent_pair.show(truncate=False)
print("Subtrate của 2 bảng trên: ")

frequent_pair_FP = frequent_pair_FP.rdd.map(lambda row: sorted(row[0])).sortBy(lambda x: x)
frequent_pair_pcy = frequent_pair_pcy.withColumn("frequent_pairs", split(frequent_pair_pcy.frequent_pairs, ", "))
frequent_pair_pcy = frequent_pair_pcy.rdd.map(lambda row: sorted(row[0])).sortBy(lambda x: x)

substrate_rdd = frequent_pair_FP.map(tuple).subtract(frequent_pair_pcy.map(tuple))
if not substrate_rdd.isEmpty():
  frequent_pair_FP.map(tuple).subtract(frequent_pair_pcy.map(tuple)).toDF().show()
print(frequent_pair_FP.map(tuple).subtract(frequent_pair_pcy.map(tuple)).take(20))

print("\t\t\t SO SÁNH ASSOCIATE RULES")
print("\n\t FPGrowth & APriori")
print("Bảng association_rules_apriori: ")
association_rules_apriori.show(truncate = False)
print("Bảng association_rules_FPGrowth: ")
association_rules.select("antecedent", "consequent", "confidence").show(truncate = False)



print("\n\t FPGrowth & PCY")
print("Bảng association_rules_pcy: ")
association_rules_pcy.show(truncate = False)
print("Bảng association_rules_FPGrowth: ")
association_rules.select("antecedent", "consequent", "confidence").show(truncate = False)

# frequent_pair.coalesce(1).write.csv('frequent_items_FP.csv', header=True)
# association_rules.coalesce(1).write.csv('association_rules_FP.csv', header=True)

Số lượng các cặp phổ biến:  302
Trong đó: 
	Số lượng cặp phổ biến size  2  là:  302

Bảng các cặp phổ biến
+----------------------------------------+----+
|items                                   |freq|
+----------------------------------------+----+
|coffee, rolls/buns                      |10  |
|coffee, bottled water                   |6   |
|coffee, yogurt                          |7   |
|coffee, shopping bags                   |8   |
|coffee, other vegetables                |6   |
|coffee, soda                            |8   |
|coffee, domestic eggs                   |5   |
|coffee, root vegetables                 |8   |
|coffee, whole milk                      |5   |
|frozen fish, whole milk                 |7   |
|specialty bar, rolls/buns               |6   |
|specialty bar, yogurt                   |5   |
|specialty bar, soda                     |5   |
|specialty bar, root vegetables          |5   |
|candy, rolls/buns                       |8   |
|candy, bottled water        